<h3> Rozważamy tweety

In [8]:
import csv

def get_data():
    return csv.reader(open("training.1600000.processed.noemoticon.csv", "rt", encoding="latin-1"))

labels = []
texts = []

for i, line in enumerate(get_data()): 
    labels.append(int(int(line[0])/4))
    texts.append(line[-1])

In [9]:
texts[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [10]:
labels[0]

0

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [12]:
len(X_train)

16000

<h2> Sieci rekurencyjne (LSTM), model do klasyfikacji

<h4> LSTM

In [5]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torchtext.vocab import GloVe
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset, DataLoader
import torchtext

tokenizer = get_tokenizer("basic_english") #duze litery będa automatyczie na małą zamienione, mozna by uzyc word_tokenize tez
glove = torchtext.vocab.GloVe(name="6B", dim=50)


#tekst -> indeksy
def text_to_indices(text):
    return [glove.stoi.get(token, 0) for token in tokenizer(text)]  # 0 = indeks dla nieznanych słów


#klasa dla danych
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.data = [torch.tensor(text_to_indices(t), dtype=torch.long) for t in texts]
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

dataset = TextDataset(X_train, y_train)

#collate_fn do obsługi paddingu i długości
def collate_batch(batch):
    sequences, labels = zip(*batch)

    #sortuje malejąco po długości (wymagane przez pack_padded_sequence)
    seq_lens = [len(seq) for seq in sequences]
    sorted_indices = sorted(range(len(seq_lens)), key=lambda i: -seq_lens[i])
    sequences = [sequences[i] for i in sorted_indices]
    labels = torch.tensor([labels[i] for i in sorted_indices])

    #paduje sekwencje do najdłuższej
    padded_seqs = pad_sequence(sequences, batch_first=True)  
    lengths = torch.tensor([len(seq) for seq in sequences])  # rzeczywiste długości

    return padded_seqs, lengths, labels

#DataLoader z naszym collate_fn i do obsługi batcha
loader = DataLoader(dataset, batch_size=250, shuffle=True, collate_fn=collate_batch)

#Wlasciwy model
class Klasyfikator_LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(glove.vectors, freeze=False)  # Wbudowane embeddingi GloVe, freee=F sprawia ze embeddingi nie będą aktualizowane
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, lengths):
        embedded = self.embedding(x) 
        packed_input = pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=True) #chowamy paddingi przed RNN
        packed_output, (hn, cn) = self.lstm(packed_input) #dane przechodzą przez LSTM
        last_hidden = hn[-1]  #bierzemy ostatni stan ukryty
        output = self.fc(last_hidden)  #klasyfikacja na podstawie ostatniego stanu ukrytego
        return output

embedding_dim = 50
hidden_size = 20
num_classes = 2

model = Klasyfikator_LSTM(embedding_dim, hidden_size, num_classes)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print("Rozpoczynam trening")
for epoch in range(15):
    model.train()
    total_loss = 0
    for batch in loader:
        inputs, lengths, targets = batch

        optimizer.zero_grad()
        outputs = model(inputs, lengths)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f}")

Rozpoczynam trening
Epoch 1 | Loss: 36.1836
Epoch 2 | Loss: 25.0502
Epoch 3 | Loss: 17.1462
Epoch 4 | Loss: 11.7116
Epoch 5 | Loss: 8.2658
Epoch 6 | Loss: 6.2544
Epoch 7 | Loss: 4.6534
Epoch 8 | Loss: 3.9906
Epoch 9 | Loss: 3.0620
Epoch 10 | Loss: 2.2881
Epoch 11 | Loss: 2.4341
Epoch 12 | Loss: 2.0429
Epoch 13 | Loss: 1.4872
Epoch 14 | Loss: 1.3573
Epoch 15 | Loss: 1.3931


<h3> Jak to wygląda na zbiorze treningowym?

In [6]:
from sklearn.metrics import accuracy_score, classification_report

# Ewaluacja modelu
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in loader:
        inputs, lengths, targets = batch
        outputs = model(inputs, lengths)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(targets.tolist())

#dokładnosć
acc = accuracy_score(all_labels, all_preds)
print(f"\n🔍 Dokładność: {acc:.4f}")

#szczegółowy raport
print("\n📊 Szczegółowy raport:")
print(classification_report(all_labels, all_preds))


🔍 Dokładność: 0.9957

📊 Szczegółowy raport:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      7981
           1       0.99      1.00      1.00      8019

    accuracy                           1.00     16000
   macro avg       1.00      1.00      1.00     16000
weighted avg       1.00      1.00      1.00     16000



<h3> Jak to wygląda na zbiorze testowym?

In [7]:
dataset2 = TextDataset(X_test, y_test)
loader2 = DataLoader(dataset2, batch_size=250, shuffle=True, collate_fn=collate_batch)


model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in loader2:
        inputs, lengths, targets = batch
        outputs = model(inputs, lengths)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(targets.tolist())

#dokładnosć
acc = accuracy_score(all_labels, all_preds)
print(f"\n🔍 Dokładność: {acc:.4f}")

#szczegółowy raport
print("\n📊 Szczegółowy raport:")
print(classification_report(all_labels, all_preds))


🔍 Dokładność: 0.7262

📊 Szczegółowy raport:
              precision    recall  f1-score   support

           0       0.73      0.72      0.73      2018
           1       0.72      0.73      0.73      1982

    accuracy                           0.73      4000
   macro avg       0.73      0.73      0.73      4000
weighted avg       0.73      0.73      0.73      4000



<h3> Inne wersje sieci rekurencyjnych

* biLSTM

self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True)

self.fc = nn.Linear(hidden_size * 2, num_classes) #bo wyjscie ma teraz wymiar 2 * hidden_size

--> W forward:

packed_output, (hn, cn) = self.lstm(packed_input)

forward_final = hn[0]

backward_final = hn[1] 

final_hidden = torch.cat((forward_final, backward_final), dim=1)

output = self.fc(final_hidden)

* GRU

self.gru = nn.GRU(embedding_dim, hidden_size, batch_first=True)  
self.fc = nn.Linear(hidden_size, num_classes)

--> w forward

packed_output, hn = self.gru(packed_input) 

last_hidden = hn[-1] 

output = self.fc(last_hidden)

<h1> SNLI

 https://nlp.stanford.edu/projects/snli/

<h4> Zadanie1 (2 pkt). Pobierz dane. Zadanie polega na klasyfikacji pary zdań (tzw. przesłanka i hipoteza) do jednej z 3 kategorii przy pomocy sieci rekurencyjnych (LSTM/biLSTM lub GRU):
    
   <br> 
    
- E (entailment, czyli z pierwszego zdania wynika drugie)
- C (contradiction, czyli drugie zdanie jest w sprzeczności z pierwszym)
- N (neutral, drugie zdanie jest neutralne w stosunku do pierwszego)
    
    <br>
   
Dla przykładu:
    
(Dzisiaj jest czwartek. Jutro jest piątek) -> E
    
(Dzisiaj jest czwartek. Jutro jest poniedziałek) -> C
    
(Dzisiaj jest czwartek. Wczoraj padało) -> N
    
    
 Uwagi i wskazówki:
- zastosuj embeddingi 50D (lub większe jeżeli będzie taka konieczność)
- w wyniku kodowania  zdań wejściowych otrzymujemy dwa wektory (wektorów) dla przesłanki i hipotezy. Te wektory najlepiej następnie skonktatenować, ewentualnie z jakimś przerywnikiem typu "implies"
- używaj batchów (np. 1000)
- wyróżnij zbiór walidacyjny celem określenia optymalnej topologii sieci
- analizuj dokładność na zbiorze treningowym i walidacyjnym co epokę (także graficznie)
- przedstaw graficznie wartość funkcji kosztu w kolejnych epokach
- potestuj różne zestawy parametrów (jak hidden size, funkcje aktywacji, dropout, batchnormalizację itd...)
- sprawdź dokładność na zbiorze testowym (przynajmniej 10% wszystkich danych) dla najlepszego modelu
- jeżeli proces trenowania będzie trwał zbyt długo to zmniejsz rozważany zbiór danych! :) Mimo wszystko finalnie powinieneś użyć pełnego zbioru (zapraszam do pracowni! :))
    
    
Dokładność na zbiorze testowym vs liczba punktów za zadanie   
* poniżej 65 --> 0.5 pkt
* 65-70 --> 1  pkt
* 70-75 --> 1.5 pkt
* ponad 75 --> 2 pkt

In [5]:
import pandas as pd

data = pd.read_json("snli_1.0/snli_1.0_train.jsonl", lines = True)
print(data.head())

  annotator_labels         captionID     gold_label               pairID  \
0        [neutral]  3416050480.jpg#4        neutral  3416050480.jpg#4r1n   
1  [contradiction]  3416050480.jpg#4  contradiction  3416050480.jpg#4r1c   
2     [entailment]  3416050480.jpg#4     entailment  3416050480.jpg#4r1e   
3        [neutral]  2267923837.jpg#2        neutral  2267923837.jpg#2r1n   
4     [entailment]  2267923837.jpg#2     entailment  2267923837.jpg#2r1e   

                                           sentence1  \
0  A person on a horse jumps over a broken down a...   
1  A person on a horse jumps over a broken down a...   
2  A person on a horse jumps over a broken down a...   
3              Children smiling and waving at camera   
4              Children smiling and waving at camera   

                              sentence1_binary_parse  \
0  ( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...   
1  ( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...   
2  ( ( ( A person ) ( on ( a horse ) )

In [74]:
data.shape

(550152, 10)

In [75]:
data["sentence1"][0]

'A person on a horse jumps over a broken down airplane.'

In [76]:
data["sentence2"][0]

'A person is training his horse for a competition.'

In [77]:
data["gold_label"][0]

'neutral'

<h2> Dostrajanie modelu (Fine-tuning)

In [9]:
from transformers import AutoTokenizer
from transformers import BertTokenizer, BertModel
from transformers import AutoModelForSequenceClassification

BERT - oparty o tzw. architekturę Transformers, jak trenowany?

Zadania:
- Masked Language Modeling (MLM) — zgadywanie brakujących słów w zdaniach,
- Next Sentence Prediction (NSP) — przewidywanie, czy zdanie B jest kontynuacją zdania A.

Przykład

* bert-base-uncased - ok 110 milionów parametrów (w tym embeddingi tokenów, parametry każdej warstwy Transformers)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") #tokenizator na modelu językowym BERT (ang) -> dobre rozumienie jezyka

Inne:
* bert-base-multilingual-cased <- wielojęzykowy
* polbert <- typowo polski, trenowany m.in. na Wikipedia
* herbert-base-case <- też polski (https://huggingface.co/allegro/herbert-base-cased)

* Dodaje warstwę do klasyfikacji

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) #buduje model klasyfikacji w oparciu o BERT

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
reviews = [
    "This product is amazing! Definitely worth buying, it meets all my expectations.",
    "Customer service was really helpful and fast, I will definitely come back again!",
    "The quality of this product is top-notch, I am very satisfied with the purchase.",
    "Everything works flawlessly, I am really happy with this transaction!",
    "I highly recommend this store, delivery was fast, and the product was exactly as described.",
    
    "Unfortunately, the product was damaged upon arrival, and customer service did not respond to my inquiry.",
    "This product completely failed to meet my expectations. I'm very disappointed with the quality.",
    "Delivery was delayed by several days, and getting in touch with the company was very difficult. I do not recommend.",
    "The product doesn't work as described, and the company offers no technical support.",
    "The website was confusing and poorly designed, making the whole buying process frustrating."
]

labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [13]:
encodings = tokenizer(reviews, truncation=True, padding=True) #przycina dluzsze teksty (modele czesto mają limit); padding dodaje 0 zeby wyrownac

In [14]:
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

dataset = TextDataset(encodings, labels)

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results", #gdzie zapisze model
    num_train_epochs=3,  #liczba epok
    per_device_train_batch_size=2, #rozmiar batcha
    logging_dir="./logs", #gdzie zapisze logi (dane z treningu)
    logging_steps=2,  #co ile batchow zapisac logi ->> mozna na bieżaco sprawdzac jak model sie uczy
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset, #mozna tez dorzucic zbior walidacyjny eval_dataset=val_dataset,
)

trainer.train()

Step,Training Loss
2,0.653600
4,0.689700
6,0.595000
8,0.524600
10,0.446000
12,0.363000
14,0.263900


TrainOutput(global_step=15, training_loss=0.48978143334388735, metrics={'train_runtime': 15.6797, 'train_samples_per_second': 1.913, 'train_steps_per_second': 0.957, 'total_flos': 385416585000.0, 'train_loss': 0.48978143334388735, 'epoch': 3.0})

In [16]:
inputs = tokenizer("Very nice product.", return_tensors="pt")
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=1)
print(predictions)  #0 lub 1

tensor([1])


In [17]:
inputs = tokenizer("Very bad product.", return_tensors="pt")
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=1)
print(predictions)  #0 lub 1

tensor([0])


<h4> Zadanie2 (2 pkt): Wybierz co najmniej 3 kandydatów na prezydenta. Stwórz zbiór co najmniej 100 wypowiedzi na osobę, a następnie zbuduj klasyfikator na bazie istniejącego modelu - w oparciu o wypowiedź tekst przewiduje który kandydat mógł to powiedzieć.
    
    
Uwagi i wskazówki:
- możesz oprzeć się na modelu herbert-base-cased 
- pobieranie wypowiedzi najlepiej zrealizuj w sposób automatyczny (np. selenium)
- może warto wybrać wypowiedzi z debat, poszukaj odpowiednich bibliotek w python do pobierania napisów z youtube; w przypadku dłuższych wypowiedzi podziel je na krótsze -> kilka zdań
- zadbaj o jakość danych! Pamiętaj, że to jest podstawa sukcesu! (dobre dane + dobry model = sukces, słabe dane + dowolony model = porażka). Usuń ewentualne duplikaty. W przypadku tweetów rozważ czy warto uwzględnić także przekierowania
- podziel dane na zbiór trenignowy/walidacyjny/testowy (7:2:1); wyznacz dokładności na tych zbiorach
- sprawdz bezpośrednio model na przykładowych wypowiedziach 
- może warto rozbudować część klasyfikacyjną, poszukaj jak to zrobić

<h4> Uzupełnienie (automatyczne pobieranie)

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

In [23]:
service = Service("C:\\Users\\Adrian\\Desktop\\Dydaktyka\\NLP\\web\\msedgedriver.exe")
browser = webdriver.Edge(service=service)

In [24]:
# Zmienna do przechowywania tweetów
tweets = []
tweets_to_scrape = 5 #ile chcemy 

url = "https://twitter.com/realdonaldtrump"
browser.get(url)


scroll_pause_time = 2  # czas na załadowanie nowych tweetów (przy przewijaniu)

while len(tweets) < tweets_to_scrape:
    tweet_elements = browser.find_elements(By.XPATH, '//article//div[@lang]')
    for tweet_element in tweet_elements:
        try:
            tweet_text = tweet_element.text
            print(tweet_text) #tego nie trzeba wyswietlac, wrzucilem tylko zeby pokazac
            tweets.append(tweet_text.replace("\n",""))
        except Exception as e:
            print(f"Cos poszlo nie tak ;/ {e}")

    if len(tweets) >= tweets_to_scrape:
        break

    #przewijanie w dół, aby załadować więcej tweetów
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)


Join me tomorrow, April 29th—in Warren, Michigan at 6:00PM Eastern!
A conversation about online child predators with Homeland Security and John Rich: PLS Repost!
Sleepy Joe Biden, THE WORST PRESIDENT IN THE HISTORY OF THE UNITED STATES, has allowed millions and millions of Criminals, many of them murderers, drug dealers, and people released from prisons and mental institutions from all around the world, to enter our Country through it’s
Following my Day One Executive Order, the Office of Personnel Management will be issuing new Civil Service Regulations for career government employees. Moving forward, career government employees, working on policy matters, will be classified as “Schedule Policy/Career,” and will
This is the hand of the man that the Democrats feel should be brought back to the United States, because he is such “a fine and innocent person.” They said he is not a member of MS-13, even though he’s got MS-13 tattooed onto his knuckles, and two Highly Respected Courts found

<h4> Uzupełnienie (regularyzacja)

W sieciach neuronowych często może dochodzić do zjawiska przetrenowania. Związane jest to m.in. z dużą swobodą w wyborze liczby parametrów. Jak temu przeciwdziałać?

- użyć więcej danych do trenowania
- zbudować mniejszą sieć (=mniej parametrów)
- dzielenie wag (weight sharing), jak np w konwolucyjnych sieciach neuronowych zastosowanie kernela dla wszystkich pozycji w obrazie
- odpowiednie wczesne zakończenie procesu uczenia (kontrolowane np poprzez zbiór walidacyjny)

Co można zrobić więcej?
- zwiększenie danych (często na podstawie istniejących przykładów)
- normalizacja danych (np BatchNormalizacja)
- uśrednianie modeli
- dropout

-> Regularyzacja L2

- zapobiega przeuczeniu, zniechęca model do dużych wartości wag

In [ ]:
torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01) 

-> Dropout

- losowo wyłączamy pewną grupę neuronów podczas uczenia (w różnych iteracjach inne), oznaczmy ten procent przez $r$ 
- powoduje uniezależnianie wag od innych,
podczas testowania korzystamy z pełni połączonej sieci (która może być traktowana jako uśrednienie tych z dropoutem), konieczna jest modyfikacja wyjścia $z$ na $z(1-r)$,
- efekt: uniezależnienie działania sieci od pojedynczych neuronów

In [ ]:
#bez
class Model(nn.Module):
    def __init__(self, n_input_features, h1, h2, n_classes):
        super().__init__()
        self.linear1 = nn.Linear(n_input_features, h1)
        self.linear2 = nn.Linear(h1, h2)
        self.linear3 = nn.Linear(h2, n_classes)

    def forward(self, x):
        l1 = self.linear1(x)
        o1 = F.relu(l1)
        l2 = self.linear2(o1)
        o2 = F.relu(l2)
        l3 = self.linear3(o2)
        return l3

In [ ]:
#z
class Model(nn.Module):
    def __init__(self, n_input_features, h1, h2, n_classes):
        super().__init__()
        self.linear1 = nn.Linear(n_input_features, h1)
        self.linear2 = nn.Linear(h1, h2)
        self.linear3 = nn.Linear(h2, n_classes)
        self.d1 = nn.Dropout(0.2) 
        self.d2 = nn.Dropout(0.2)
    def forward(self, x):
        o1 = F.relu(self.linear1(x))
        o1 = self.d1(o1)
        o2 = F.relu(self.linear2(o1))
        o2 = self.d2(o2)
        o3 = self.linear3(o2)
    return o3


-> Normalizacja

- Cel: ustabilizowanie i przyspieszenie procesu uczenia przez normalizację w każdym "batchu", stosujemy najczęściej przed funkcją aktywacji, stabilizuje zakres wartości

In [ ]:
class Model(nn.Module):
    def __init__(self, n_input_features, h1, h2, n_classes):
        super().__init__()
        self.linear1 = nn.Linear(n_input_features, h1)
        self.bn1 = nn.BatchNorm1d(h1)
        self.d1 = nn.Dropout(0.2)

        self.linear2 = nn.Linear(h1, h2)
        self.bn2 = nn.BatchNorm1d(h2)
        self.d2 = nn.Dropout(0.2)

        self.linear3 = nn.Linear(h2, n_classes)

    def forward(self, x):
        o1 = self.linear1(x)
        o1 = self.bn1(o1)
        o1 = F.relu(o1)
        o1 = self.d1(o1)

        o2 = self.linear2(o1)
        o2 = self.bn2(o2)
        o2 = F.relu(o2)
        o2 = self.d2(o2)

        o3 = self.linear3(o2)
        return o3